In [ ]:
# To run this you need to have all the required files defined in outputkeys.py
import simtool as st
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
from typing import List, Dict, Any,Union,Tuple
#from Visulizer_Class import DataVisualizerSim2l as dv
from outputkeys import paths
import os


In [38]:
# Gets the simtool name and the inputs 

simToolName = "nemo1dsim2l.ipynb"
nb = st.utils.searchForSimTool(simToolName)
inputs = st.utils.getSimToolInputs(nb)



In [39]:
### Visualizer Class taken from dash web app data vizulization and modified to work with ipywidgets


class DataVisualizer:
        def __init__(self, path):
            
                self.PATH = path
                # Get list of available JSON files
                if os.path.exists(path):
                    self.json_files = [f for f in os.listdir(self.PATH) if f.endswith('.json')]
                else:
                    self.json_files = []
                
                self.current_data = {}
                self.log_y = False
                self.log_x = False
                self.swap_axes = False
                self.combined_view = False
                self.unique_voltages = []
                self.file_titles = {}  # Store titles for each file separately
        
        def load_data(self, selected_files):
            self.current_data = {}
            self.combined_view = False
            
            # Check if combined view is selected
            if "BND+IV+TRAN+" in selected_files:
                self.combined_view = True
                # Find BND, IV, and TRAN files
                bnd_files = [f for f in self.json_files if "bnd" in f.lower()]
                iv_files = [f for f in self.json_files if "nd_iv.json" in f.lower()]
                tran_files = [f for f in self.json_files if any(x in f.lower() for x in ["nd_tran.json", "nd_j_int_e.json", "nd_curd.json"])]
                
                # Use the first file from each category
                selected_files = []
                if bnd_files:
                    selected_files.append(bnd_files[0])
                if iv_files:
                    selected_files.append(iv_files[0])
                if tran_files:
                    selected_files.extend(tran_files[:3])  # Get up to 3 TRAN files
            
            for selected_file in selected_files:
                if selected_file != "BND+IV+TRAN+":
                    try:
                        file_path = f'{self.PATH}/{selected_file}'
                        if os.path.exists(file_path):
                            with open(file_path, 'r') as file:
                                lines = file.readlines()
                                if len(lines) < 2:
                                    continue
                                
                                key_from_second_line = lines[1].split(":")[0].strip().strip('"')
                                file_title = key_from_second_line.replace("_", " ").replace("data file", " ")
                                
                                # Store title for this specific file
                                self.file_titles[selected_file] = file_title
                                
                                try:
                                    data = json.loads(''.join(lines))
                                    if key_from_second_line in data:
                                        self.current_data[selected_file] = data[key_from_second_line]
                                except json.JSONDecodeError:
                                    continue
                    except Exception as e:
                        print(f"Error loading file {selected_file}: {e}")
            
            if self.current_data:
                # Check if voltage is on x-axis using the first file
                first_file = list(self.current_data.keys())[0]
                if self.current_data[first_file] and len(self.current_data[first_file]) > 0:
                    first_entry = self.current_data[first_file][0]
                    if "x_data" in first_entry and "label" in first_entry["x_data"]:
                        first_x_label = str(first_entry["x_data"]["label"]).lower()
                        self.use_slider = "voltage" not in first_x_label
                    else:
                        self.use_slider = True
                else:
                    self.use_slider = True
                
                if self.use_slider:
                    # Extract voltages from all entries
                    all_voltages = []
                    for data in self.current_data.values():
                        for entry in data:
                            if "voltage" in entry:
                                all_voltages.append(entry["voltage"])
                    
                    self.unique_voltages = sorted(set(all_voltages))

        def create_figure(self, current_voltage=0, log_y=False, log_x=False, swap_axes=False):
            # Check if combined view is selected
            if hasattr(self, 'combined_view') and self.combined_view:
                return self.create_combined_figure(current_voltage, log_y, log_x, swap_axes)
            
            # Regular single plot view
            fig = go.Figure()
            fig.update_layout(template='plotly_white')
            
            xtitle = ""
            ytitle = ""
            
            # Get title from current file (first file if multiple)
            first_file = list(self.current_data.keys())[0] if self.current_data else ""
            plot_title = self.file_titles.get(first_file, 'Data Visualization') if hasattr(self, 'file_titles') else 'Data Visualization'
        
            if hasattr(self, 'use_slider') and self.use_slider and self.unique_voltages:
                closest_voltage = min(self.unique_voltages, key=lambda x: abs(x - current_voltage))
                for file, data in self.current_data.items():
                    filtered_data = [entry for entry in data if entry.get("voltage") == closest_voltage]
                    for entry in filtered_data:
                        if swap_axes:
                            x_data = entry["y_data"]["data"]
                            y_data = entry["x_data"]["data"]
                            xtitle = str(entry["y_data"]["label"])
                            ytitle = str(entry["x_data"]["label"])
                        else:
                            x_data = entry["x_data"]["data"]
                            y_data = entry["y_data"]["data"]
                            xtitle = str(entry["x_data"]["label"])
                            ytitle = str(entry["y_data"]["label"])
                        
                        min_length = min(len(x_data), len(y_data))
                        x_data = x_data[:min_length]
                        y_data = y_data[:min_length]
                        
                        fig.add_trace(go.Scatter(
                            x=x_data,
                            y=y_data,
                            mode="lines",
                            name=f"{entry.get('legend', file)}"
                        ))
            else:
                for file, data in self.current_data.items():
                    combined_x = []
                    combined_y = []
                    for entry in data:
                        if "x_data" in entry and "y_data" in entry:
                            if len(entry["x_data"]["data"]) > 0 and len(entry["y_data"]["data"]) > 0:
                                if swap_axes:
                                    x_point = entry["y_data"]["data"][0]
                                    y_point = entry["x_data"]["data"][0]
                                    xtitle = str(entry["y_data"]["label"])
                                    ytitle = str(entry["x_data"]["label"])
                                else:
                                    x_point = entry["x_data"]["data"][0]
                                    y_point = entry["y_data"]["data"][0]
                                    xtitle = str(entry["x_data"]["label"])
                                    ytitle = str(entry["y_data"]["label"])
                                
                                combined_x.append(x_point)
                                combined_y.append(y_point)
                    
                    if combined_x and combined_y:
                        fig.add_trace(go.Scatter(
                            x=combined_x,
                            y=combined_y,
                            mode="lines+markers",
                            name=f"{file}"
                        ))
            
            fig.update_layout(
                title=plot_title,
                xaxis_title=xtitle,
                yaxis_title=ytitle,
                xaxis=dict(type='log' if log_x else 'linear'),
                yaxis=dict(type='log' if log_y else 'linear'),
                height=500
            )
                
            return fig





class DataVisualizerSim2l(DataVisualizer):
        def __init__(self, outputs_dict: Dict[str, str], r):
            """
            Initialize the DataVisualizerSim2l with a dictionary of outputs and a sim2l object.
            
            :param outputs_dict: Dictionary mapping output identifiers to their file names
            :param r: A sim2l object that has a read method to access cached data
            """
            # Store the sim2l object
            self.r = r
            
            # Store the outputs dictionary
            self.outputs_dict = outputs_dict
            
            # Initialize state
            self.current_data = {}
            self.log_y = False
            self.log_x = False
            self.swap_axes = False
            self.is_playing = False
            self.unique_voltages = []
            
            # Get available outputs by checking which ones return data via r.read
            self.available_outputs = self.get_available_outputs()
            
            # Create widgets
            self.create_widgets()
            
            # Load initial data if any outputs are available
            if self.available_outputs:
                self.load_data([self.available_outputs[0]])
        

        def contains_legend(self,data):
        # Check if 'legend' exists in any nested dictionary (non-recursively)
            if isinstance(data, dict):
                for key, value in data.items():
                    # If the value is a list, check each item in the list
                    if isinstance(value, list):
                        for item in value:
                            if isinstance(item, dict) and "legend" in item:
                                print(f"Found legend in {key}: {item}")  # Debugging print
                                return True
                    # If the value is a dictionary and contains 'legend'
                    elif isinstance(value, dict) and "legend" in value:
                        return True
            return False



        def get_available_outputs(self):
            """
            Check which outputs are available by trying to read them with r.read
            
            :return: List of available output names
            """
            available = []
            for name in self.outputs_dict:
                data = self.r.read(name)


                # If none of the above conditions are true, then add to available
                if data is not None and self.contains_legend(data):
                    available.append(name)
                    print(f"Successfully found data for: {name}")
                    print(self.contains_legend(data))
                else:
                    print(f"No data available for: {name}")
            
            return available
            
        def create_widgets(self):
            # File selection dropdown (using available outputs)
            self.file_dropdown = widgets.SelectMultiple(
                options=self.available_outputs,
                description='Outputs:',
                style={'description_width': 'initial'}
            )
            
            # Voltage slider (will be updated when data is loaded)
            self.voltage_slider = widgets.FloatSlider(
                value=0,
                min=0,
                max=0,
                description='Voltage:',
                style={'description_width': 'initial'},
                continuous_update=True,
                readout_format='.3f'
            )
            
            # Control buttons
            self.y_log_button = widgets.ToggleButton(
                value=False,
                description='Toggle Y Log Scale',
                style={'description_width': 'initial'}
            )
            
            self.x_log_button = widgets.ToggleButton(
                value=False,
                description='Toggle X Log Scale',
                style={'description_width': 'initial'}
            )
            
            self.swap_axes_button = widgets.ToggleButton(
                value=False,
                description='Swap Axes',
                style={'description_width': 'initial'}
            )
            
            self.play_button = widgets.ToggleButton(
                value=False,
                description='Play Animation',
                style={'description_width': 'initial'}
            )
            
            # Output widget for the plot
            self.output = widgets.Output()
            
            # Register callbacks
            self.file_dropdown.observe(self.on_file_change, names='value')
            self.voltage_slider.observe(self.update_plot, names='value')
            self.y_log_button.observe(self.update_plot, names='value')
            self.x_log_button.observe(self.update_plot, names='value')
            self.swap_axes_button.observe(self.update_plot, names='value')
            self.play_button.observe(self.toggle_animation, names='value')
        
        # Override the load_data method to use the sim2l object
        def load_data(self, selected_outputs: List[str]):
            """
            Load data for the selected outputs using the sim2l object's read method
            
            :param selected_outputs: List of output identifiers to load
            """
            self.current_data = {}
            
            for output_name in selected_outputs:
                # Get data directly from the sim2l object
                data = self.r.read(output_name)
                
                if data is not None:
                    # Assuming the data is a dictionary with a key that contains the actual data
                    if isinstance(data, dict):
                        # Find the main data key - assuming it's the first key
                        # that contains the underscore-separated file name
                        for key in data:
                            if output_name.split('_')[-1] in key or key.endswith("data file"):
                                self.current_data[output_name] = data[key]
                                # Store title for this output
                                self.title = key.replace("_", " ").replace("data file", " ")
                                break
                        else:
                            # If no matching key found, use the first key
                            key = next(iter(data))
                            self.current_data[output_name] = data[key]
                            self.title = key.replace("_", " ").replace("data file", " ")
            
            # If no data was loaded, exit early
            if not self.current_data or not any(self.current_data.values()):
                return
                
            # Check for voltage slider usage by examining the first entry
            first_output = next(iter(self.current_data))
            first_data = self.current_data[first_output]
            
            if not first_data:
                # No data points available
                self.use_slider = False
                self.voltage_slider.layout.visibility = 'hidden'
                return
                
            first_entry = first_data[0]
            if "x_data" in first_entry and "label" in first_entry["x_data"]:
                first_x_label = str(first_entry["x_data"]["label"]).lower()
                self.use_slider = "voltage" not in first_x_label
            else:
                self.use_slider = True
        
            if self.use_slider:
                # Extract all voltage values from the loaded data
                all_voltages = []
                for data in self.current_data.values():
                    for entry in data:
                        if "voltage" in entry:
                            all_voltages.append(entry["voltage"])
                
                if all_voltages:
                    self.unique_voltages = sorted(set(all_voltages))
                    self.voltage_slider.min = min(self.unique_voltages)
                    self.voltage_slider.max = max(self.unique_voltages)
                    self.voltage_slider.value = self.unique_voltages[0]
                    self.voltage_slider.step = (self.unique_voltages[1] - self.unique_voltages[0]) if len(self.unique_voltages) > 1 else 1
                    self.voltage_slider.disabled = False
                    self.voltage_slider.layout.visibility = 'visible'
                else:
                    self.voltage_slider.disabled = True
                    self.voltage_slider.layout.visibility = 'hidden'
            else:
                self.voltage_slider.disabled = True
                self.voltage_slider.layout.visibility = 'hidden'
        
        # Override create_figure to use widget values
        def create_figure(self) -> go.Figure:
            # Call parent method with parameters from widgets
            return super().create_figure(
                current_voltage=self.voltage_slider.value if hasattr(self, 'voltage_slider') else 0,
                log_y=self.y_log_button.value if hasattr(self, 'y_log_button') else False,
                log_x=self.x_log_button.value if hasattr(self, 'x_log_button') else False,
                swap_axes=self.swap_axes_button.value if hasattr(self, 'swap_axes_button') else False
            )
        
        def update_plot(self, change):
            with self.output:
                clear_output(wait=True)
                fig = self.create_figure()
                display(fig)
        
        def on_file_change(self, change):
            self.load_data(change.new)
            self.update_plot(None)
        
        def toggle_animation(self, change):
            self.is_playing = change.new
            if self.is_playing and self.use_slider:
                self.animate()
        
        async def animate(self):
            import asyncio
            while self.is_playing and self.use_slider:
                current_idx = self.unique_voltages.index(min(self.unique_voltages, key=lambda x: abs(x - self.voltage_slider.value)))
                next_idx = (current_idx + 1) % len(self.unique_voltages)
                self.voltage_slider.value = self.unique_voltages[next_idx]
                await asyncio.sleep(1)
        
        def display(self):
            # Create layout with plot at the top and controls at the bottom
            controls = widgets.HBox([
                self.y_log_button,
                self.x_log_button,
                self.swap_axes_button,
                self.play_button
            ])
            
            bottom_controls = widgets.VBox([
                self.file_dropdown,
                self.voltage_slider,
                controls
            ])
            
            layout = widgets.VBox([
                self.output,
                bottom_controls
            ])
            
            display(layout)
            self.update_plot(None)

In [40]:
# GUI Variables to be passed to the simtool
from ipywidgets import HBox, VBox, HTML, Image, Layout, Button, ButtonStyle, Tab, BoundedIntText, BoundedFloatText, Dropdown, ToggleButton, Output, Textarea, FloatLogSlider



Energy_Band_Model = Dropdown(

    description = 'Energy Band  Model',
    value = 'Effective Mass Model',
    options=['Effective Mass Model', 'Tight-Binding Sp3s* Model', 'Tight-Binding sp3d5s* Model'],
    style = {'description_width': 'initial'},
    layout=Layout(width='450px'),
    tooltip = 'Teset'
)


Potential_model =  Dropdown(

    description = 'Potential Model',
    value = 'Thomas-Fermi',
     options=['Thomas-Fermi', 'Hartree'],
    style = {'description_width': 'initial'}
)


Ending_Bias = BoundedIntText(
    min = 0,
    max = 1000,
    description = 'Ending Bias (V)',
    value = 0,
    style = {'description_width': 'initial'}
)

Device = Dropdown(
   description = 'Device Type',
    value = '1-Barrier-Device',
    options=['1-Barrier-Device', '2-Barrier-Device', '3-Barrier-Device', '6-Barrier-Device'],
    style = {'description_width': 'initial'}
)


points = BoundedFloatText(
    min = 0.0,
    max = 100.0,
    description = 'No. of Points',
    value = 6,
    style = {'description_width': 'initial'}
)

Starting_bias = BoundedFloatText(
    min = 0.0,
    max = 100.0,
    value = 0,
    description = 'Starting Bias',
    style = {'description_width': 'initial'},
    tooltip = 'Test'
)

Temperature = BoundedFloatText(
    min = 77.0,
    max = 5500.0,
    description = 'Temperature (K)',
    value = 300,
    style = {'description_width': 'initial'},
    tooltip="Select the model for energy band representation"
    

)



SC_ChargeRegion = BoundedIntText(
    min = 0,
    description = 'Semiclassical Charge Region dSC (nm)',
    value = 0,
    style = {'description_width': 'initial'}
)

EQ_Region = BoundedIntText(
    min =0,
    description = 'Equilibrium Region dEQ (nm)',
    value = 0,
    style = {'description_width': 'initial'}
)



Quantum_charge = Dropdown(
    description = 'Quantum Charge',
    value = False,
    options = [True,False],
    style = {'description_width': 'initial'},
    tooltip ='Charge'
)

Lattice_Constant = BoundedIntText(
    
    description = 'Lattice Constant (nm)',
    value = 0.283,
    style = {'description_width': 'initial'}
)

Not_Normalized_Current_Plot = Dropdown(
    description = 'Not-Normalized Current Plot',
    value = False,
    options = [True, False],
    style = {'description_width': 'initial'}
)

Scatter_Plot = Dropdown(
    description = 'Scatter Plot',
    value = False,
    options = [True, False],
    style = {'description_width': 'initial'}
)

Carrier_Surface_Distrubution_Plot = Dropdown(
    description = 'Carrier Surface Distribution Plot',
    value = False,
    options = [True, False],
    style = {'description_width': 'initial'}
)


Resivoir_Relaxation_Model = Dropdown(
    description = 'Resivoir Relaxation Model',
    value = 'Energy Independant',
    options = ['Energy Independant','Exponentialy Damped', 'Lorentzian Damped'],
    style = {'description_width': 'initial'}
)


Resivoir_Relaxation_Energy = BoundedIntText(
    min =0,
    description = 'Resivoir Relaxation Energy (meV)',
    value = 0,
    style = {'description_width': 'initial'}
)

Decay_Length= BoundedIntText(
    min =0,
    description = 'Decay Length (meV)',
    value = 0,
    style = {'description_width': 'initial'}
)

Resonance_Finder = Dropdown(
    description = 'Resonance Finder',
    value = False,
    options = ['True',False],
    style = {'description_width': 'initial'}
)


No_Homogenous_Grid_Points= BoundedIntText(
    min =0,
    description = 'No. Homogenous Grid Points',
    value = 0,
    style = {'description_width': 'initial'}
)

No_Points_per_res= BoundedIntText(
    min =0,
    description = 'No. Points Per Resonance',
    value = 0,
    style = {'description_width': 'initial'}
)

No_Points_per_EC_EF= BoundedIntText(
    min =0,
    description = 'No. Points per EC EF',
    value = 0,
    style = {'description_width': 'initial'}

)

Lancos_Iteration_Step_Size= BoundedIntText(
    min =0,
    description = 'Lancos Iteration Step Size (eV)',
    value = 0,
    style = {'description_width': 'initial'}
)


Lancos_Iteration_Limit= BoundedIntText(
    min =0,
    description = 'Lancos Iteration Limit',
    value = 0,
    style = {'description_width': 'initial'}
)

Newton_Iteration_Step_Size= BoundedIntText(
    min =0,
    description = 'Newton Iteration Step Size (eV)',
    value = 0,
    style = {'description_width': 'initial'}
)


Newton_Solver_Convergance_Condition= BoundedIntText(
    description = 'Newton Solver Convergance Condition (eV)',
    value = 0,
    style = {'description_width': 'initial'}
)



w_output = Output()

outputContainer = VBox()

In [41]:
# This is the run function that will get all the inputs from the user to the simtool
# Also creates the visualizer object that will plot the outputs

r = None

def RunST():
    w_output.clear_output()
    with w_output:
        global r
        # Update inputs with values from the new widgets
        inputs.Energy_Band_Model.value = Energy_Band_Model.value
        inputs.Potential_model.value = Potential_model.value
        inputs.Ending_Bias.value = Ending_Bias.value
        inputs.Device.value = Device.value
        inputs.points.value = points.value
        inputs.Starting_bias.value = Starting_bias.value
        inputs.TEMPERATURE.value = Temperature.value
        inputs.SC_ChargeRegion.value = SC_ChargeRegion.value
        inputs.EQ_Region.value = EQ_Region.value
        inputs.Quantum_charge.value = Quantum_charge.value
        inputs.Lattice_Constant.value = Lattice_Constant.value
        #inputs.Not_Normalized_Current_Plot.value = Not_Normalized_Current_Plot.value
        inputs.Scatter_Plot.value = Scatter_Plot.value
        inputs.Carrier_Surface_Distrubution_Plot.value = Carrier_Surface_Distrubution_Plot.value
        inputs.Resivoir_Relaxation_Model.value = Resivoir_Relaxation_Model.value
        inputs.Resivoir_Relaxation_Energy.value = Resivoir_Relaxation_Energy.value
        inputs.Decay_Length.value = Decay_Length.value
        inputs.Resonance_Finder.value = Resonance_Finder.value
        inputs.No_Homogenous_Grid_Points.value = No_Homogenous_Grid_Points.value
        inputs.No_Points_per_res.value = No_Points_per_res.value
        inputs.No_Points_per_EC_EF.value = No_Points_per_EC_EF.value
        inputs.Lancos_Iteration_Step_Size.value = Lancos_Iteration_Step_Size.value
        inputs.Lancos_Iteration_Limit.value = Lancos_Iteration_Limit.value
        inputs.Newton_Iteration_Step_Size.value = Newton_Iteration_Step_Size.value
        #inputs.Newton_Solver_Convergance_Condition.value = Newton_Solver_Convergance_Condition.value Current Bug this is no working 
        print(inputs)
        r = st.Run(nb, inputs,cache=False)
        # These are the ouputs as viewed by the user
        # Right now we are just showing the raw ouput from the simulator 
        # In the future want to map each output to this in the outputkeys file and autopopulate the dropdown menu
        '''outputDisplay = Dropdown(
            description = "Display Output",
            options=[
                
                "Bands+Transmission+CurrentDensity+IV",
                "ConductionBand+CurrentDensity+Resonances",
                "Bands+Transmission+CurrentDensity+IV+Charge",
                "Bands+Transmission+CurrentDensity+IV+Charge (Zoomed Energy Range)",
                "Bands+Transmission+CurrentDensity",
                "Current Voltage Characteristic",
                "Conduction Band",
                "J_INT_E",
                "j_int_e_av",
                "J_INT_K",
                "J_K",
                "J_PHI",
                "KJ_INT_K",
                "KJ_K",
                "RES",
                "KN_XK_INT_K",
                "N_SHEET",
                "N_XK",
                "RESENERGY_V",
                "RESTRANS_E",
                "RESTRANS_V",
                "RESWIDTH_V",
                "Transmission Coefficient",
                "CurrentDensity+NormalizedCumulativeCurrentDensity",
                "Transmission+CurrentDensity+NormalizedCumulativeCurrentDensity",
                "Charge Densities",
                "Current Density",
                "Sheet Density",
                "Wavefunction Magnitude Squared",
                "Wavefunction",
                "Resonance Energy Vs. Voltage",
                "Resonance Energy+Width vs Voltage",
                "Material and Data Structure Table",
                "Input Deck",
                "Output Log"
            ],
            style = {'description_width': 'initial'},
            value = "Current Voltage Characteristic"
        )
        outputDisplay.observe(lambda b : DisplayOutput(r, b['new'],w_output), 'value')
        outputContainer.children=[outputDisplay]
        DisplayOutput(r, outputDisplay.value, w_output)
        '''
     
        visualizer = DataVisualizerSim2l(outputs_dict=paths,r=r)
        

        
        visualizer.display()

       

In [42]:
#### GUI for User Inputs ####

    

# Create the button and the event handler for running the simulation
run_But = Button(
    description="Simulate ...", 
    tooltip='Run simulation', 
    icon='check', 
    disable=False,
    layout=Layout(width='auto'),
    style=ButtonStyle(button_color='lightblue')
)

run_But.on_click(lambda b: RunST())  # Define the action for the button click

# Create the always visible widgets section (Device and Energy Band Model)
always_visible_widgets = VBox(
    children=[
        Device,  
        Energy_Band_Model  
    ],
)

# Define the parameters for each tab

parameters_basic = [
    Temperature,
    Starting_bias,
    Ending_Bias,
    points,
    Potential_model,
    run_But
]

parameters_domains = [
    SC_ChargeRegion,
    EQ_Region,
    run_But
]

parameters_advanced = [    
    Not_Normalized_Current_Plot,
    Scatter_Plot,
    Carrier_Surface_Distrubution_Plot,
    Resivoir_Relaxation_Model,
    Resivoir_Relaxation_Model,
    Resivoir_Relaxation_Energy,
    Decay_Length,
    run_But
]

parameters_resfind = [    
    Resonance_Finder,
    No_Homogenous_Grid_Points,
    No_Points_per_res,
    No_Points_per_EC_EF,
    Lancos_Iteration_Step_Size,
    Lancos_Iteration_Limit,
    Newton_Iteration_Step_Size,
    Newton_Solver_Convergance_Condition,
    run_But
]


    
    
    

# Combine all parameter lists into one
parameters = parameters_basic + parameters_domains + parameters_advanced + parameters_resfind

# Container for the structure of the simulation widgets
container_structure = VBox(layout=Layout(width='auto', height='100%', border='1px solid rgba(0, 0, 1,0 )'))

# Set the children for the different tabs in the GUI
container_materials = VBox()
container_materials.children = parameters_domains
container_structure.children = parameters_basic
container_ambient = VBox()
container_ambient.children = parameters_advanced
container_resfind = VBox()
container_resfind.children = parameters_resfind


# Create the Tabs widget with the different sections
Tabs = Tab(
    layout=Layout(
        width='800px',  # Change Depending on the size of the window
    )
)

ptab=widgets.Tab()
ptab.children= [widgets.Output() for _ in range(3)]
ptab.set_title(0, "Dimension")
ptab.set_title(1, "Doping")
ptab.set_title(2, "Mole Fraction X")
Tabs.children = [container_structure, container_materials, container_ambient,container_resfind]
Tabs.set_title(0, "Basic")
Tabs.set_title(1, "Multiscale Domains")
Tabs.set_title(2, "Advanced")
Tabs.set_title(3, "Resonance Finder")

# Create the output container widget for simulation results


output_cont = VBox(layout=Layout(height='auto', width='auto ', border='1px solid rgba(0, 0, 0, 1)'))
output_cont.children = [w_output]  


# Create the options container (always visible widgets + tabs)
opt = VBox()
opt.children = [always_visible_widgets, Tabs]

properties=VBox()


# Create the layout for the whole application
main_layout = HBox(layout=Layout(width='1000px'))  # Use HBox to align widgets horizontally
main_layout.children = [opt,properties]

all = VBox(layout=Layout(height='auto', width='auto', border='1px solid rgba(0, 0, 0, 1)'))
all.children = [main_layout,output_cont,outputContainer,run_But]


# Display the layout
display(all)



